In [ ]:
# ASL Number Detector (0–5)
import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hand.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7)


def count_fingers(lms):
    finger_tips = [8,12,16,20]
    count = 0

    if lms[4][0] > lms[3][0]:
        count+=1

    for tip in finger_tips:
        if lms[tip][1] < lms[tip-2][1]:
            count+=1

    return count


while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)
    h,w,_ = rgb.shape

    lms = []
    if res.multi_hand_landmarks:
        for hand_markdowns in res.multi_hand_landmarks:
            for id,lm in enumerate(hand_markdowns.landmark):
                cx,cy = int(lm.x*w),int(lm.y*h)
                lms.append((cx,cy))
            mp_drawing.draw_landmarks(f,hand_markdowns,mp_hand.HAND_CONNECTIONS)
    # print(len(lms))
    if len(lms)>=21:
        fingers = count_fingers(lms)
        cv2.putText(f,f"ASL Number : {fingers}",(10,100),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),2)



    cv2.imshow("ASL Number Detector",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()